# Task 1: Preprocess and Explore the Data

## Objective
Load, clean, and understand the data to prepare it for modeling.

This notebook covers:
1. Extracting historical financial data using YFinance
2. Data cleaning and understanding
3. Exploratory Data Analysis (EDA)
4. Seasonality and Trend Analysis
5. Risk Metrics Calculation

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style for better visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## 1. Extract Historical Financial Data

We'll fetch data for three assets:
- **TSLA** (Tesla): High-growth stock
- **BND** (Vanguard Total Bond Market ETF): Low risk bonds
- **SPY** (S&P 500 ETF): Moderate risk market index

Period: January 1, 2015 to January 15, 2026

In [ ]:
# Define assets and date range
tickers = ['TSLA', 'BND', 'SPY']
start_date = '2015-01-01'
end_date = '2026-01-15'

# Fetch data for all assets
print("Fetching historical data from YFinance...")
data_dict = {}

for ticker in tickers:
    print(f"Downloading {ticker}...")
    stock = yf.Ticker(ticker)
    df = stock.history(start=start_date, end=end_date)
    df['Ticker'] = ticker
    data_dict[ticker] = df
    print(f"{ticker}: {len(df)} records downloaded")

print("\nData extraction complete!")

In [ ]:
# Combine all dataframes
combined_data = pd.concat(data_dict.values(), ignore_index=False)
combined_data = combined_data.reset_index()

# Display first few rows
print("Combined Dataset Shape:", combined_data.shape)
print("\nFirst few rows:")
combined_data.head()

## 2. Data Cleaning and Understanding

In [ ]:
# Check data types
print("Data Types:")
print(combined_data.dtypes)
print("\n" + "="*50)

# Check for missing values
print("\nMissing Values:")
missing_values = combined_data.isnull().sum()
print(missing_values[missing_values > 0])
if missing_values.sum() == 0:
    print("No missing values found!")

print("\n" + "="*50)
# Basic statistics
print("\nBasic Statistics:")
combined_data.describe()

In [ ]:
# Ensure Date column is datetime type
combined_data['Date'] = pd.to_datetime(combined_data['Date'])

# Set Date as index for time series analysis
combined_data = combined_data.set_index('Date')

# Sort by date
combined_data = combined_data.sort_index()

# Check date range for each ticker
print("Date Range for each asset:")
for ticker in tickers:
    ticker_data = combined_data[combined_data['Ticker'] == ticker]
    print(f"{ticker}: {ticker_data.index.min()} to {ticker_data.index.max()} ({len(ticker_data)} days)")

# Handle any remaining missing values (if any)
if combined_data.isnull().sum().sum() > 0:
    print("\nFilling missing values...")
    # Forward fill for missing values (common in financial data)
    combined_data = combined_data.ffill()
    # If still missing, backward fill
    combined_data = combined_data.bfill()
    
print("\nData cleaning complete!")

In [ ]:
# Create separate dataframes for each asset for easier analysis
tsla_data = combined_data[combined_data['Ticker'] == 'TSLA'].copy()
bnd_data = combined_data[combined_data['Ticker'] == 'BND'].copy()
spy_data = combined_data[combined_data['Ticker'] == 'SPY'].copy()

# Calculate daily returns for each asset
for df, name in [(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]:
    df['Daily_Return'] = df['Close'].pct_change()
    df['Log_Return'] = np.log(df['Close'] / df['Close'].shift(1))

print("Daily returns calculated for all assets")

## 3. Exploratory Data Analysis (EDA)

### 3.1 Closing Price Over Time

In [ ]:
# Visualization 1: Closing prices over time
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

for idx, (df, ticker, title) in enumerate([(tsla_data, 'TSLA', 'Tesla (TSLA)'),
                                           (bnd_data, 'BND', 'Vanguard Total Bond Market ETF (BND)'),
                                           (spy_data, 'SPY', 'S&P 500 ETF (SPY)')]):
    axes[idx].plot(df.index, df['Close'], linewidth=1.5, label=f'{ticker} Close Price')
    axes[idx].set_title(f'{title} - Closing Price Over Time', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Date', fontsize=12)
    axes[idx].set_ylabel('Price (USD)', fontsize=12)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].legend()

plt.tight_layout()
plt.savefig('../data/processed/closing_prices_over_time.png', dpi=300, bbox_inches='tight')
plt.show()

print("Key Insights:")
print("- TSLA shows high volatility and significant growth over the period")
print("- BND shows relatively stable prices with gradual upward trend")
print("- SPY shows moderate volatility with overall upward trend")

### 3.2 Daily Percentage Change (Returns)

In [ ]:
# Visualization 2: Daily returns
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

for idx, (df, ticker, title) in enumerate([(tsla_data, 'TSLA', 'Tesla (TSLA)'),
                                           (bnd_data, 'BND', 'Vanguard Total Bond Market ETF (BND)'),
                                           (spy_data, 'SPY', 'S&P 500 ETF (SPY)')]):
    axes[idx].plot(df.index, df['Daily_Return'] * 100, linewidth=0.8, alpha=0.7, label=f'{ticker} Daily Returns')
    axes[idx].axhline(y=0, color='r', linestyle='--', linewidth=1)
    axes[idx].set_title(f'{title} - Daily Percentage Returns', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Date', fontsize=12)
    axes[idx].set_ylabel('Daily Return (%)', fontsize=12)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].legend()

plt.tight_layout()
plt.savefig('../data/processed/daily_returns.png', dpi=300, bbox_inches='tight')
plt.show()

# Statistics on daily returns
print("Daily Returns Statistics:")
print("="*60)
for df, ticker in [(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]:
    returns = df['Daily_Return'].dropna()
    print(f"\n{ticker}:")
    print(f"  Mean: {returns.mean()*100:.4f}%")
    print(f"  Std Dev: {returns.std()*100:.4f}%")
    print(f"  Min: {returns.min()*100:.4f}%")
    print(f"  Max: {returns.max()*100:.4f}%")
    print(f"  Skewness: {returns.skew():.4f}")
    print(f"  Kurtosis: {returns.kurtosis():.4f}")

### 3.3 Volatility Analysis (Rolling Statistics)

In [ ]:
# Calculate rolling statistics (30-day and 60-day windows)
window_short = 30
window_long = 60

for df in [tsla_data, bnd_data, spy_data]:
    df['Rolling_Mean_30'] = df['Close'].rolling(window=window_short).mean()
    df['Rolling_Std_30'] = df['Close'].rolling(window=window_short).std()
    df['Rolling_Mean_60'] = df['Close'].rolling(window=window_long).mean()
    df['Rolling_Std_60'] = df['Close'].rolling(window=window_long).std()
    df['Rolling_Volatility_30'] = df['Daily_Return'].rolling(window=window_short).std() * np.sqrt(252) * 100  # Annualized

# Visualization 3: Rolling volatility
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

for idx, (df, ticker, title) in enumerate([(tsla_data, 'TSLA', 'Tesla (TSLA)'),
                                           (bnd_data, 'BND', 'Vanguard Total Bond Market ETF (BND)'),
                                           (spy_data, 'SPY', 'S&P 500 ETF (SPY)')]):
    axes[idx].plot(df.index, df['Rolling_Volatility_30'], linewidth=1.5, label='30-Day Rolling Volatility (Annualized)')
    axes[idx].set_title(f'{title} - Rolling Volatility (30-Day Window)', fontsize=14, fontweight='bold')
    axes[idx].set_xlabel('Date', fontsize=12)
    axes[idx].set_ylabel('Volatility (%)', fontsize=12)
    axes[idx].grid(True, alpha=0.3)
    axes[idx].legend()

plt.tight_layout()
plt.savefig('../data/processed/rolling_volatility.png', dpi=300, bbox_inches='tight')
plt.show()

print("Volatility Insights:")
print("- TSLA shows the highest volatility, with significant spikes during market events")
print("- BND shows the lowest and most stable volatility")
print("- SPY shows moderate volatility, typically between TSLA and BND")

### 3.4 Outlier Detection

In [ ]:
# Detect outliers using IQR method
def detect_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers

# Detect outliers in daily returns
print("Outlier Detection in Daily Returns (IQR Method):")
print("="*60)
for df, ticker in [(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]:
    outliers = detect_outliers_iqr(df, 'Daily_Return')
    print(f"\n{ticker}: {len(outliers)} outliers detected ({len(outliers)/len(df)*100:.2f}% of data)")
    if len(outliers) > 0:
        print(f"  Most extreme positive return: {outliers['Daily_Return'].max()*100:.2f}% on {outliers.loc[outliers['Daily_Return'].idxmax()].name}")
        print(f"  Most extreme negative return: {outliers['Daily_Return'].min()*100:.2f}% on {outliers.loc[outliers['Daily_Return'].idxmin()].name}")

# Visualization: Box plots for daily returns
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, (df, ticker) in enumerate([(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]):
    axes[idx].boxplot(df['Daily_Return'].dropna() * 100, vert=True)
    axes[idx].set_title(f'{ticker} - Daily Returns Distribution', fontsize=14, fontweight='bold')
    axes[idx].set_ylabel('Daily Return (%)', fontsize=12)
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/processed/returns_boxplot.png', dpi=300, bbox_inches='tight')
plt.show()

### 3.5 Analyze Days with Unusually High or Low Returns

In [ ]:
# Identify extreme return days (top 5% and bottom 5%)
print("Extreme Return Days Analysis:")
print("="*60)

for df, ticker in [(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]:
    returns = df['Daily_Return'].dropna()
    threshold_high = returns.quantile(0.95)
    threshold_low = returns.quantile(0.05)
    
    extreme_high = df[df['Daily_Return'] >= threshold_high].sort_values('Daily_Return', ascending=False).head(10)
    extreme_low = df[df['Daily_Return'] <= threshold_low].sort_values('Daily_Return', ascending=True).head(10)
    
    print(f"\n{ticker} - Top 10 Highest Returns:")
    print(extreme_high[['Close', 'Daily_Return']].to_string())
    
    print(f"\n{ticker} - Top 10 Lowest Returns:")
    print(extreme_low[['Close', 'Daily_Return']].to_string())

## 4. Seasonality and Trend Analysis

### 4.1 Augmented Dickey-Fuller (ADF) Test for Stationarity

In [ ]:
from statsmodels.tsa.stattools import adfuller

def adf_test(timeseries, title="Time Series"):
    """
    Perform Augmented Dickey-Fuller test for stationarity
    """
    print(f'\n{"="*60}')
    print(f'Augmented Dickey-Fuller Test: {title}')
    print(f'{"="*60}')
    
    adf_result = adfuller(timeseries.dropna(), autolag='AIC')
    
    print(f'ADF Statistic: {adf_result[0]:.6f}')
    print(f'p-value: {adf_result[1]:.6f}')
    print(f'Number of Lags Used: {adf_result[2]}')
    print(f'Number of Observations: {adf_result[3]}')
    
    print('\nCritical Values:')
    for key, value in adf_result[4].items():
        print(f'   {key}: {value:.6f}')
    
    if adf_result[1] <= 0.05:
        print(f'\n✓ Result: Series is STATIONARY (p-value <= 0.05)')
        print('  → No differencing required for ARIMA models')
    else:
        print(f'\n✗ Result: Series is NON-STATIONARY (p-value > 0.05)')
        print('  → Differencing required for ARIMA models (d > 0)')
    
    return adf_result

# Test closing prices (typically non-stationary)
print("STATIONARITY TEST: Closing Prices")
print("="*70)
for df, ticker in [(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]:
    adf_test(df['Close'], f'{ticker} Closing Price')

In [ ]:
# Test daily returns (typically stationary)
print("\n\nSTATIONARITY TEST: Daily Returns")
print("="*70)
for df, ticker in [(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]:
    adf_test(df['Daily_Return'], f'{ticker} Daily Returns')

### 4.2 Interpretation of Stationarity Results

**Key Findings:**
- **Closing Prices**: Typically non-stationary (p-value > 0.05), indicating trends and requiring differencing (d > 0) for ARIMA models
- **Daily Returns**: Typically stationary (p-value ≤ 0.05), indicating mean-reverting behavior suitable for ARIMA modeling

**Implications for Modeling:**
- For ARIMA models on prices: We'll need to difference the data (d parameter)
- For ARIMA models on returns: We can use d=0 or d=1
- LSTM models can handle non-stationary data but may benefit from normalization

## 5. Calculate Risk Metrics

### 5.1 Value at Risk (VaR)

In [ ]:
from scipy import stats

def calculate_var(returns, confidence_level=0.05):
    """
    Calculate Value at Risk (VaR) using historical method
    VaR represents the maximum expected loss at a given confidence level
    """
    # Historical VaR: percentile of returns
    var_historical = np.percentile(returns.dropna(), confidence_level * 100)
    
    # Parametric VaR: assuming normal distribution
    mean_return = returns.mean()
    std_return = returns.std()
    var_parametric = stats.norm.ppf(confidence_level, mean_return, std_return)
    
    return var_historical, var_parametric

print("Value at Risk (VaR) - 95% Confidence Level (5% VaR)")
print("="*70)
print("VaR represents the maximum expected loss with 95% confidence")
print("="*70)

for df, ticker in [(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]:
    returns = df['Daily_Return'].dropna()
    var_hist, var_param = calculate_var(returns, confidence_level=0.05)
    
    print(f"\n{ticker}:")
    print(f"  Historical VaR (5%): {var_hist*100:.4f}%")
    print(f"  Parametric VaR (5%): {var_param*100:.4f}%")
    print(f"  Interpretation: We expect losses to exceed {abs(var_hist*100):.2f}% on 5% of trading days")

### 5.2 Sharpe Ratio

In [ ]:
def calculate_sharpe_ratio(returns, risk_free_rate=0.02, periods_per_year=252):
    """
    Calculate Sharpe Ratio: (Mean Return - Risk-Free Rate) / Std Dev of Returns
    Annualized Sharpe Ratio
    """
    mean_return = returns.mean() * periods_per_year  # Annualized
    std_return = returns.std() * np.sqrt(periods_per_year)  # Annualized
    
    # Annualized risk-free rate
    rf_annual = risk_free_rate
    
    sharpe_ratio = (mean_return - rf_annual) / std_return
    
    return sharpe_ratio, mean_return, std_return

print("Sharpe Ratio (Risk-Adjusted Returns)")
print("="*70)
print("Assumes 2% annual risk-free rate")
print("Higher Sharpe Ratio = Better risk-adjusted returns")
print("="*70)

for df, ticker in [(tsla_data, 'TSLA'), (bnd_data, 'BND'), (spy_data, 'SPY')]:
    returns = df['Daily_Return'].dropna()
    sharpe, ann_return, ann_vol = calculate_sharpe_ratio(returns)
    
    print(f"\n{ticker}:")
    print(f"  Annualized Return: {ann_return*100:.2f}%")
    print(f"  Annualized Volatility: {ann_vol*100:.2f}%")
    print(f"  Sharpe Ratio: {sharpe:.4f}")
    
    if sharpe > 1:
        interpretation = "Excellent"
    elif sharpe > 0.5:
        interpretation = "Good"
    elif sharpe > 0:
        interpretation = "Acceptable"
    else:
        interpretation = "Poor"
    print(f"  Interpretation: {interpretation} risk-adjusted returns")

## 6. Summary of Key Insights

### Data Quality
- All data successfully extracted from YFinance
- No missing values detected
- Data types properly formatted
- Date range: January 1, 2015 to January 15, 2026

### Key Patterns Identified
1. **TSLA**: High volatility, significant growth, extreme returns
2. **BND**: Low volatility, stable prices, consistent returns
3. **SPY**: Moderate volatility, steady growth, market-representative returns

### Stationarity
- Closing prices are non-stationary (require differencing for ARIMA)
- Daily returns are stationary (suitable for ARIMA with d=0 or d=1)

### Risk Profile
- TSLA: Highest risk (highest VaR, variable Sharpe ratio)
- BND: Lowest risk (lowest VaR, stable returns)
- SPY: Moderate risk (balanced risk-return profile)

In [ ]:
# Save processed data for Task 2
tsla_data.to_csv('../data/processed/TSLA_processed.csv')
bnd_data.to_csv('../data/processed/BND_processed.csv')
spy_data.to_csv('../data/processed/SPY_processed.csv')
combined_data.to_csv('../data/processed/combined_data.csv')

print("Processed data saved to ../data/processed/")
print("Files saved:")
print("  - TSLA_processed.csv")
print("  - BND_processed.csv")
print("  - SPY_processed.csv")
print("  - combined_data.csv")